#### Please note this notebook is only for educational purpose. The code is adapted from https://github.com/jimmysong/programmingbitcoin .

In [1]:
from random import randint
from hashlib import sha256

class FieldElement:
    def __init__(self, num, prime):
        if num >= prime or num < 0:
            raise ValueError("Number {0} is not in the field range 0 to {1}".format(num, prime - 1))
        self.num = num
        self.prime = prime
        
    def __eq__(self, other):
        if not other:
            return False
        elif self.num == other.num and self.prime == other.prime:
            return True
        else:
            return False
    
    def __ne__(self, other):
        if not other:
            return False
        elif self.num != other.num or self.prime != other.prime:
            return True
        else:
            return False
        
    def __repr__(self):
        return 'FieldElement_{0}({1})'.format(self.prime, self.num)
    
    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num + other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot subtract two numbers in different fields.")
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot multiply two numbers in different fields")
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __pow__(self, exponent):
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)
    
    def __truediv__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot divide two numbers in different fields.")
        num = self.num * pow(other.num, other.prime - 2, other.prime) % self.prime
        return self.__class__(num, self.prime)
    
    def __rmul__(self, coefficient):
        num = (coefficient * self.num) % self.prime
        return self.__class__(num, self.prime)
    
class Point:
    def __init__(self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y**2 != self.x**3 + a * x + b:
            raise ValueError("({0}, {1}) is not on the curve".format(x, y))
    
    def __repr__(self):
        if self.x is None and self.y is None:
            return 'Point(infinity)'
        elif isinstance(self.x, FieldElement):
            return 'Point({0},{1})_{2}_{3} FieldElement({4})'.format(
                self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return "Point({0},{1})_{2}_{3}".format(self.x, self.y, self.a, self.b)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.a == other.a and self.b == other.b
    
    def __ne__(self, other):
        return self.x != other.x or self.y != other.y or self.a != other.a or self.b != other.b
    
    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError("Points {0}, {1} are not on the same curve".format(self, other))
        
        if self.x is None:
            return other
        
        if other.x is None:
            return self
        
        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)
        
        if self.x != other.x:
            s = (other.y - self.y)/(other.x - self.x)
            x = s**2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)
        
        if self == other:
            if self.y == 0 * self.x:
                return self.__class__(None, None, self.a, self.b)
            else:
                s = (3 * self.x**2 + self.a)/(2 * self.y)
                x = s**2 - 2 * self.x
                y = s * (self.x - x) - self.y
                return self.__class__(x, y, self.a, self.b)
    
    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result

class S256Field(FieldElement):
    def __init__(self, num, prime = None):
        super().__init__(num = num, prime = P)
    
    def __repr__(self):
        return '{:x}'.format(self.num).zfill(64)
    
class S256Point(Point):
    def __init__(self, x, y, a = None, b = None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(x = S256Field(x), y = S256Field(y), a = a, b = b)
        else:
            super().__init__(x = x, y = y, a = a, b = b)
    
    def __repr__(self):
        if self.x is None and self.y is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({}, {})'.format(self.x, self.y)
    
    def __rmul__(self, coefficient):
        coef = coefficient % N
        return super().__rmul__(coef)
    
    def verify(self, z, sig):
        s_inv = pow(sig.s, N - 2, N)
        u = z * s_inv % N
        v = sig.r * s_inv % N
        total = u * G + v * self
        return total.x.num == sig.r

### Define secp256k1 ###
A = 0
B = 7
P = 2**256 - 2**32 - 977
X = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
Y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
G = S256Point(X, Y)

class Signature:
    def __init__(self, r, s):
        self.r = r
        self.s = s
    
    def __repr__(self):
        return 'Signature({:x},{:x})'.format(self.r, self.s)
    
class PrivateKey:
    def __init__(self, secret):
        self.secret = secret
        self.point = secret * G # knowing point, hard to know secret, aka discrete log problem
        
    def hex(self):
        return '{:x}'.format(self.secret).zfill(64)
    
    def sign(self, z):
        k = randint(0, N)
        r = (k*G).x.num
        k_inv = pow(k, N-2, N)
        s = (z + r*self.secret) * k_inv % N
        return Signature(r, s)

def hash_message(message):
    return int.from_bytes(sha256(sha256(message.encode('utf-8')).digest()).digest(), "big")

In [2]:
private_key1 = PrivateKey(randint(0, N))
public_key1 = private_key1.point
private_key2 = PrivateKey(randint(0, N))
public_key2 = private_key2.point

In [3]:
message = "Hello World!"
hashed_message = hash_message(message)
signature1 = private_key1.sign(hashed_message)

In [4]:
public_key1.verify(hashed_message, signature1)

True

In [5]:
public_key2.verify(hashed_message, signature1)

False